# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [32]:
import pandas as pd
import numpy as np
import os
import shutil
# from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [33]:
stock_df = pd.DataFrame()

files_m = os.listdir('../data/test_label/')

for min_df in files_m[:1]:
    _df = pd.read_csv(f'../data/test_label/{min_df}', index_col='Unnamed: 0')
    stock_df = pd.concat([stock_df, _df])
    print(min_df)

label_066970_엘앤에프.csv


In [34]:
# 첫 5줄을 봅니다.
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률,label
20221103,1530,216500,216500,216500,216500,0,12129,2625928000,210922,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,216500
20221103,1520,216800,216900,216700,216800,0,863,187072000,210922,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,216900
20221103,1519,216900,216900,216700,216800,0,1106,239789000,210524,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,216900
20221103,1518,216900,216900,216800,216800,0,461,99981000,210091,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,216900
20221103,1517,216600,216900,216600,216900,0,601,130274000,209993,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,216900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201029,905,37509,37557,37365,37461,0,1387,51942000,5773,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,37702
20201029,904,37413,37509,37317,37461,0,1810,67708000,4798,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,37702
20201029,903,37413,37461,37269,37461,0,2672,99978000,4464,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,37702
20201029,902,37221,37413,36980,37413,0,5894,219176000,3571,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,37702


In [35]:
def data_type_trans(stock_df):

    stock_df = stock_df.astype({'시간':'uint16'})
    stock_df = stock_df.astype({'시가':'uint32'})
    stock_df = stock_df.astype({'고가':'uint32'})
    stock_df = stock_df.astype({'저가':'uint32'})
    stock_df = stock_df.astype({'종가':'uint32'})
    stock_df = stock_df.astype({'전일대비':'int32'})
    stock_df = stock_df.astype({'거래량':'uint32'})
    stock_df = stock_df.astype({'거래대금':'uint64'})
    stock_df = stock_df.astype({'누적체결매도수량':'uint64'})
    stock_df = stock_df.astype({'상장주식수':'uint64'})
    stock_df = stock_df.astype({'시가총액':'uint64'})
    stock_df = stock_df.astype({'외국인주문한도수량':'uint64'})
    stock_df = stock_df.astype({'외국인주문가능수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유비율':'float32'})
    stock_df = stock_df.astype({'수정주가일자':'uint32'})
    stock_df = stock_df.astype({'수정주가비율':'float32'})
    stock_df = stock_df.astype({'기관순매수량':'int64'})
    stock_df = stock_df.astype({'기관누적순매수량':'int64'})
    stock_df = stock_df.drop(['등락주선', '등락비율', '예탁금', '주식회전율', '거래성립률'], axis=1)
    stock_df = stock_df.astype({'label':'uint32'})

    stock_df.info()

    return stock_df

In [36]:
stock_df = data_type_trans(stock_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189537 entries, 20221103 to 20201029
Data columns (total 20 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   시간         189537 non-null  uint16 
 1   시가         189537 non-null  uint32 
 2   고가         189537 non-null  uint32 
 3   저가         189537 non-null  uint32 
 4   종가         189537 non-null  uint32 
 5   전일대비       189537 non-null  int32  
 6   거래량        189537 non-null  uint32 
 7   거래대금       189537 non-null  uint64 
 8   누적체결매도수량   189537 non-null  uint64 
 9   상장주식수      189537 non-null  uint64 
 10  시가총액       189537 non-null  uint64 
 11  외국인주문한도수량  189537 non-null  uint64 
 12  외국인주문가능수량  189537 non-null  uint64 
 13  외국인현보유수량   189537 non-null  uint64 
 14  외국인현보유비율   189537 non-null  float32
 15  수정주가일자     189537 non-null  uint32 
 16  수정주가비율     189537 non-null  float32
 17  기관순매수량     189537 non-null  int64  
 18  기관누적순매수량   189537 non-null  int64  
 19  label      189

In [37]:
trans = stock_df.loc[:,['고가','label']]
trans = trans.rename(columns={'label':'pct_label'}).T

In [38]:
get_trans = trans.pct_change().T.iloc[:,-1]
get_trans

20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.000000
              ...   
20201029    0.003861
20201029    0.005145
20201029    0.006433
20201029    0.007725
20201029    0.009019
Name: pct_label, Length: 189537, dtype: float64

In [39]:
stock_df = pd.concat([stock_df, get_trans], axis=1)
stock_df = stock_df.drop(['label'], axis=1)
stock_df



,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,216500,216500,216500,216500,0,12129,2625928000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1520,216800,216900,216700,216800,0,863,187072000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1519,216900,216900,216700,216800,0,1106,239789000,210524,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1518,216900,216900,216800,216800,0,461,99981000,210091,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1517,216600,216900,216600,216900,0,601,130274000,209993,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201029,905,37509,37557,37365,37461,0,1387,51942000,5773,0,0,0,0,0,0.0,0,0.0,0,0,0.003861
20201029,904,37413,37509,37317,37461,0,1810,67708000,4798,0,0,0,0,0,0.0,0,0.0,0,0,0.005145
20201029,903,37413,37461,37269,37461,0,2672,99978000,4464,0,0,0,0,0,0.0,0,0.0,0,0,0.006433
20201029,902,37221,37413,36980,37413,0,5894,219176000,3571,0,0,0,0,0,0.0,0,0.0,0,0,0.007725


In [40]:
stock_df['pct_label'] = stock_df['pct_label'].mul(100)
stock_df['pct_label'] = stock_df['pct_label'].round(0)
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,216500,216500,216500,216500,0,12129,2625928000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1520,216800,216900,216700,216800,0,863,187072000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1519,216900,216900,216700,216800,0,1106,239789000,210524,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1518,216900,216900,216800,216800,0,461,99981000,210091,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1517,216600,216900,216600,216900,0,601,130274000,209993,0,0,0,0,0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201029,905,37509,37557,37365,37461,0,1387,51942000,5773,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20201029,904,37413,37509,37317,37461,0,1810,67708000,4798,0,0,0,0,0,0.0,0,0.0,0,0,1.0
20201029,903,37413,37461,37269,37461,0,2672,99978000,4464,0,0,0,0,0,0.0,0,0.0,0,0,1.0
20201029,902,37221,37413,36980,37413,0,5894,219176000,3571,0,0,0,0,0,0.0,0,0.0,0,0,1.0


In [41]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
stock_df = stock_df.drop(columns=['상장주식수', '시가총액', '외국인주문한도수량', '외국인주문가능수량', '외국인현보유수량', '외국인현보유비율', '수정주가일자', '수정주가비율', '기관순매수량', '기관누적순매수량']
            , axis=1
)

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]


In [42]:
from sklearn.model_selection import train_test_split

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
X_stock_trans_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189537 entries, 0 to 189536
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시간        189537 non-null  float64
 1   시가        189537 non-null  float64
 2   고가        189537 non-null  float64
 3   저가        189537 non-null  float64
 4   종가        189537 non-null  float64
 5   전일대비      189537 non-null  float64
 6   거래량       189537 non-null  float64
 7   거래대금      189537 non-null  float64
 8   누적체결매도수량  189537 non-null  float64
dtypes: float64(9)
memory usage: 13.0 MB


In [43]:
y_stock_df.value_counts()

0.0     86287
1.0     59683
2.0     22115
3.0     10275
4.0      4913
5.0      2860
6.0      1328
7.0       701
9.0       310
8.0       297
11.0      293
10.0      210
12.0       74
27.0       34
28.0       33
13.0       24
25.0       20
26.0       19
15.0       16
16.0       12
17.0        6
24.0        6
23.0        5
14.0        3
22.0        3
18.0        3
21.0        2
20.0        2
29.0        2
19.0        1
Name: pct_label, dtype: int64

In [44]:
y_stock_df = to_categorical(y_stock_df)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_stock_sc, y_stock_df, test_size=0.2, shuffle=True, random_state=42)

In [46]:
# X_stock_trans_df 데이터 변환 해줘야 함
X_stock_trans_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량
0,0.862826,0.743855,0.741962,0.744830,0.743855,0.0,0.007660,0.007394,0.044146
1,0.849108,0.745096,0.743613,0.745659,0.745096,0.0,0.000544,0.000527,0.044146
2,0.847737,0.745510,0.743613,0.745659,0.745096,0.0,0.000698,0.000675,0.044063
3,0.846365,0.745510,0.743613,0.746073,0.745096,0.0,0.000291,0.000281,0.043972
4,0.844993,0.744269,0.743613,0.745245,0.745510,0.0,0.000379,0.000367,0.043952
...,...,...,...,...,...,...,...,...,...
189532,0.005487,0.003182,0.003175,0.002788,0.002984,0.0,0.000875,0.000146,0.001204
189533,0.004115,0.002785,0.002977,0.002589,0.002984,0.0,0.001142,0.000191,0.001000
189534,0.002743,0.002785,0.002779,0.002390,0.002984,0.0,0.001687,0.000281,0.000930
189535,0.001372,0.001990,0.002580,0.001193,0.002785,0.0,0.003722,0.000617,0.000743


In [47]:
X_train

array([[8.23045267e-03, 7.86890673e-01, 7.84899179e-01, ...,
        2.20477295e-03, 2.22881129e-03, 2.67014928e-03],
       [1.50891632e-02, 2.05362907e-01, 2.05489406e-01, ...,
        5.15983814e-03, 1.99019300e-03, 8.04289320e-03],
       [4.59533608e-01, 6.85508566e-01, 6.83748121e-01, ...,
        4.91984567e-04, 4.44236719e-04, 2.61892746e-02],
       ...,
       [4.37585734e-01, 2.14114872e-01, 2.13428732e-01, ...,
        1.47153279e-04, 5.81129836e-05, 1.96763375e-02],
       [2.77091907e-01, 2.29239427e-01, 2.28518818e-01, ...,
        1.95151773e-04, 8.02499223e-05, 3.37647451e-02],
       [5.88477366e-01, 3.47016469e-01, 3.46027447e-01, ...,
        1.88583558e-03, 1.01281985e-03, 1.77202585e-01]])

In [50]:
# 레이블 카테고리 개수
y_stock_df.shape

(189537, 30)

In [17]:
y_stock_df.value_counts()

0.0     86287
1.0     59683
2.0     22115
3.0     10275
4.0      4913
5.0      2860
6.0      1328
7.0       701
9.0       310
8.0       297
11.0      293
10.0      210
12.0       74
27.0       34
28.0       33
13.0       24
25.0       20
26.0       19
15.0       16
16.0       12
17.0        6
24.0        6
23.0        5
14.0        3
22.0        3
18.0        3
21.0        2
20.0        2
29.0        2
19.0        1
Name: pct_label, dtype: int64

## 3. 원-핫 인코딩

In [18]:
# # 원-핫 인코딩 처리를 합니다.
# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)

## 4. 소프트맥스

In [19]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM

# # 모델 설정
# model = Sequential()
# model.add(Dense(1000,  input_dim=X_stock_df.shape[1], activation='relu'))
# model.add(LSTM(X_stock_df.shape[1], activation='tanh'))
# model.add(Dense(len(np.max(y_train)), activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
# model.summary()

# # 모델 컴파일
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # 모델 실행
# history=model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_test, y_test))

# model.evaluate(X_test, y_test)[1]

In [20]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [21]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

X_train_lstm = X_train.reshape(X_train.shape[0], 9, 1)
X_test_lstm = X_test.reshape(X_test.shape[0], 9, 1)


In [52]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
print(y_train.shape)
print(X_train_lstm.shape)
print(X_train_lstm)

(151629, 30)
(151629, 9, 1)
[[[8.23045267e-03]
  [7.86890673e-01]
  [7.84899179e-01]
  ...
  [2.20477295e-03]
  [2.22881129e-03]
  [2.67014928e-03]]

 [[1.50891632e-02]
  [2.05362907e-01]
  [2.05489406e-01]
  ...
  [5.15983814e-03]
  [1.99019300e-03]
  [8.04289320e-03]]

 [[4.59533608e-01]
  [6.85508566e-01]
  [6.83748121e-01]
  ...
  [4.91984567e-04]
  [4.44236719e-04]
  [2.61892746e-02]]

 ...

 [[4.37585734e-01]
  [2.14114872e-01]
  [2.13428732e-01]
  ...
  [1.47153279e-04]
  [5.81129836e-05]
  [1.96763375e-02]]

 [[2.77091907e-01]
  [2.29239427e-01]
  [2.28518818e-01]
  ...
  [1.95151773e-04]
  [8.02499223e-05]
  [3.37647451e-02]]

 [[5.88477366e-01]
  [3.47016469e-01]
  [3.46027447e-01]
  ...
  [1.88583558e-03]
  [1.01281985e-03]
  [1.77202585e-01]]]


In [25]:
y_train.shape

(151629, 30)

In [30]:
y_test.shape

(37908, 29)

In [54]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [55]:

# 모델 설정
model = Sequential()
model.add(LSTM(100, input_shape=(9, 1), activation='tanh'))
model.add(Dense(30, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train_lstm, y_train, epochs=10, batch_size=200,verbose=1)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
Total params: 43,830
Trainable params: 43,830
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
759/759 [==============================] - 9s 9ms/step - loss: 1.4122 - accuracy: 0.4522
Epoch 2/10
759/759 [==============================] - 7s 9ms/step - loss: 1.3287 - accuracy: 0.4527
Epoch 3/10
759/759 [==============================] - 7s 9ms/step - loss: 1.3141 - accuracy: 0.4522
Epoch 4/10
759/759 [==============================] - 7s 9ms/step - loss: 1.3052 - accuracy: 0.4524
Epoch 5/10
759/759 [==========================

In [56]:
model.evaluate(X_test, y_test)[1]

1185/1185 [==============================] - 3s 2ms/step - loss: 1.2840 - accuracy: 0.4674


0.46736836433410645

In [57]:
predict = model.predict(X_test)

1185/1185 [==============================] - 3s 2ms/step


In [58]:
predict[0]

array([4.2029354e-01, 4.4462919e-01, 9.3305081e-02, 3.0655138e-02,
       3.7626575e-03, 2.6620491e-03, 9.4436173e-04, 8.2237145e-04,
       6.0316158e-04, 1.1170356e-03, 6.5218250e-04, 4.9251720e-04,
       5.1227478e-05, 6.0500128e-07, 6.5825922e-08, 6.8021411e-09,
       7.4221584e-09, 1.8849461e-08, 3.1931517e-08, 2.8315020e-08,
       8.0168796e-08, 5.0661203e-08, 4.7603056e-08, 2.3916846e-07,
       2.4148812e-07, 7.5432155e-08, 1.1267751e-06, 5.7122579e-06,
       1.0238116e-06, 1.6246007e-07], dtype=float32)

In [ ]:
predict.index(predict[0][0].max())

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [60]:
# for i,rate  in enumerate(predict):
#     for i, ra in enumerate(rate):
#         if rate.max() == ra:
#             print(i)

print(list(map(np.argmax, predict)))

[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 